# КОМАНДА 21 (Deep Seers)
### В этом ноутбуке представлено наше решение задачи классификации изображений для соревнования ML intensive Yandex Academy autumn 2024

## ВВЕДЕНИЕ


### Когда мы начали работу над задачей, первым делом мы внимательно изучили предоставленный датасет. Мы загрузили данные и проанализировали их структуру, чтобы понять, какие классы представлены и как они распределены. Это позволило нам оценить, есть ли какие-либо классы, которые могут быть недопредставлены, что могло бы повлиять на качество модели.
### Паралелльно нами были прочитаны несколько статей, котрорые определили наш первоначальный вектор развития:
[Одна из статей](https://yfalan.github.io/files/papers/Xipeng_CVPR2018.pdf) 
> !!!!Основываясь на статье, мы использовали:
RandomRotation для поворота изображений,  
RandomHorizontalFlip для добавления симметричности, 
RandomErasing для случайного закрытия областей изображения, 
RandomResizedCrop для масштабирования изображений.!!!!
### После, мы распредили обязанности: два участника команды занимались пробой различных архитектур, один занимался аугментациями для датасета.
### Мы создали [репозиторий на github](http://github.com/bottic/CV_project_yandex) и выделили каждому участнику по воркспейсу


## ХОД РАБОТЫ

### Через несколько дней был готов полностью рабочий для обучения [ноутбук](https://github.com/bottic/CV_project_yandex/blob/0b79254887288665e84e9a428c5f81c171143bc7/VladSpace/project.ipynb), в нем мы имели:
* Загрузку датасета
* Подготовку изображений для обучения модели
* Функции для обучения модели
* Сама базовая модель
* Отрисовку графиков для отслеживания успехов модели
### Код для загрузки датасета, использовали API kaggle

In [ ]:
# Скачиваем датасет
! pip install -q kaggle # Предпологается что kaggle.json уже в этой папке
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c ml-intensive-yandex-academy-autumn-2024
# Распаковка и удаление зипа
! unzip ml-intensive-yandex-academy-autumn-2024.zip && rm ml-intensive-yandex-academy-autumn-2024.zip

### Также был создан кастомный класс для датасета

In [ ]:
class HumanPoseDataset(Dataset):
    def __init__(self, img_dir, csv_file, transform=None):
        """
        img_dir: Папка с изображениями (В моем случае, 'drive/MyDrive/DataSets/human_poses_data/img_train').
        csv_file: Путь к таблице с метками (например, 'train_answers.csv').
        transform: Трансформации для предобработки изображений.
        """
        self.img_dir = img_dir
        self.labels = pd.read_csv(csv_file)  # Загружаем таблицу меток
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Достаем имя изображения и метку
        img_id = self.labels.iloc[idx, 0]  # img_id (имя изображения)
        label = self.labels.iloc[idx, 1]  # target_feature (метка)

        # Загружаем изображение
        img_path = os.path.join(self.img_dir, str(img_id)+'.jpg')
        image = Image.open(img_path).convert("RGB")  # Убедимся, что изображение в RGB

        # Применяем трансформации
        if self.transform:
            image = self.transform(image)

        return image, label


### Функция для просмотра изображений из кастомного класса датасета

In [ ]:
def show_images_with_labels(dataset, id_to_category, num_images=10):
    fig, axes = plt.subplots(1, num_images, figsize=(20, 5))
    for i in range(num_images):
        # Достаем изображение и метку
        image, label = dataset[i]

        # Декодируем метку в категорию
        category = id_to_category[label] if id_to_category else label
        axes[i].imshow(image.permute(1, 2, 0))  

### Так как мы сохраняли метрики в csv файл можно было удобно смотреть на графики обучения при помощи этого кода

In [ ]:
metrics_path = "./train_info/logs_13_12_21_19/metrics.csv"

metrics = pd.read_csv(metrics_path)

plt.figure()
metrics[['Train Loss', 'Validation Loss', 'Validation Accuracy']].plot()

### Первой архитектурой, которую мы попробывали был [Alexnet](https://github.com/bottic/CV_project_yandex/blob/1f63ff09077e41349f18877b5ef845273540522f/VladSpace/project.ipynb). Мы выбрали эту модель, так как она является одной из классических архитектур для задач классификации изображений и показала хорошие результаты на различных датасетах.

### Мы начали с предобученной версии AlexNet, чтобы использовать трансферное обучение и ускорить процесс обучения. Это позволило нам извлечь полезные признаки из изображений, которые были уже изучены на большом наборе данных, таком как ImageNet. Мы адаптировали модель под наш датасет, изменив последний слой, чтобы он соответствовал количеству классов в нашей задаче.

### В процессе обучения мы использовали стандартные гиперпараметры, такие как скорость обучения, размер батча и количество эпох. Однако, несмотря на наши усилия, максимальная точность, которую мы смогли достичь на валидационном наборе, составила всего 0.5. Это было разочаровывающим результатом, особенно учитывая, что мы ожидали более высокую производительность от такой известной архитектуры.

### Мы заметили, что начиная с 20-й эпохи, модель начала переобучаться. Это проявлялось в том, что точность на обучающем наборе продолжала расти, в то время как точность на валидационном наборе начала снижаться. Мы решили, что необходимо внести изменения в подход к обучению, чтобы улучшить обобщающую способность модели.

### В качестве первых шагов по борьбе с переобучением мы начали экспериментировать с различными техниками регуляризации, такими как Dropout и L2-регуляризация.
### Итоговая первая вразумительная версия модели выглядела так:

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=20):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

### Еще была и такая вариация модели:

In [ ]:
class AlexNetArtem(nn.Module):
    def __init__(self, num_classes):
        super(AlexNetArtem, self).__init__()
        self.num_classes = num_classes
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(
                in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=0
            ),
            nn.ReLU(),
            nn.LocalResponseNorm(
                alpha=1e-4, beta=0.75, k=2, size=5
            ),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=96, out_channels=256, kernel_size=5, padding=2
            ),
            nn.ReLU(),
            nn.LocalResponseNorm(
                alpha=1e-4, beta=0.75, k=2, size=5
            ),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(
                in_channels=256, out_channels=384, kernel_size=3, padding=1
            ),
            nn.ReLU(),
        )
        self.conv_block_4 = nn.Sequential(
            nn.Conv2d(
                in_channels=384, out_channels=384, kernel_size=3, padding=1
            ),
            nn.ReLU(),
        )
        self.conv_block_5 = nn.Sequential(
            nn.Conv2d(
                in_channels=384, out_channels=256, kernel_size=3, padding=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=256 * 6 * 6, out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(
                in_features=4096, out_features=self.num_classes
            ),
        )
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.conv_block_3(x)
        x = self.conv_block_4(x)
        x = self.conv_block_5(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

### Паралелльно мы создали модель на основе архитектуры **ResNet18**, изначально она показывала схожие с нашей первоночальной моделью, но после некоторых изменений в архитектуре, включая такие как:
* Активационная функция Mish
* Групповая нормализация
* SE-блоки
* Дропаут
### Модель стала давать accuracy около 0.62
*Стоит сказать, что мы обучали модели в одинаковых условиях с функцией потерь в виде кросс-энтропии, закрепленным random seed и одинаковой по количеству val и train выборке*

In [ ]:
import torch
import torch.nn as nn

class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(nn.functional.softplus(x))

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        y = x.view(b, c, -1).mean(dim=2)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).view(b, c, 1, 1)
        return x * y

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None, dropout_prob=0.3):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.mish = Mish()
        self.group_norm1 = nn.GroupNorm(32, out_channels)
        self.dropout1 = nn.Dropout2d(p=dropout_prob)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.group_norm2 = nn.GroupNorm(32, out_channels)
        self.dropout2 = nn.Dropout2d(p=dropout_prob)

        self.se_block = SEBlock(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.mish(out)
        out = self.group_norm1(out)
        out = self.dropout1(out)

        out = self.conv2(out)
        out = self.mish(out)
        out = self.group_norm2(out)
        out = self.dropout2(out)

        out = self.se_block(out)
        out += identity
        out = self.mish(out)
        return out

class UpdatedResNet(nn.Module):
    def __init__(self, block, layers, num_classes=20, dropout_prob=0.3):
        super(UpdatedResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.mish = Mish()
        self.group_norm = nn.GroupNorm(32, 64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0], dropout_prob=dropout_prob)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dropout_prob=dropout_prob)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dropout_prob=dropout_prob)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dropout_prob=dropout_prob)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1, dropout_prob=0.3):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.GroupNorm(32, out_channels)
            )
        
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample, dropout_prob=dropout_prob))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels, dropout_prob=dropout_prob))
        
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.mish(x)
        x = self.group_norm(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

def custom_resnet18(num_classes=20, dropout_prob=0.3):
    return UpdatedResNet(ResidualBlock, [2, 2, 2, 2], num_classes, dropout_prob)

### Вскоре мы сделали аугментацию данных, дабы повысить обобщающую способность модели. Датасет был раздут до 19 тыс. изображений.
### Код для создания двух папок с аугментированными изображениями и изобажениями для валидации

In [ ]:
class SplitPFileToVal:
    def __init__(self, path_to_dir, path_to_new_dir_val, path_to_new_dir_train, p:float):
        """
        Создаёт 2 новые папки: Для валидации - % от всех изображений, для трейна - все остальное. 
        Нужно для отделения части датасета, для val данных

        path_to_dir: Папка с изображениями. (img_train)
        path_to_new_dir_val: Путь до новой папки, можно не создавать вручную.
        path_to_new_dir_train: Путь до новой папки, можно не создавать вручную.
        p: % изображений, которые будут отделены для валидационного датасета
        """

        #Проверка, введеного %
        if not isinstance(p, float):
            raise TypeError("% должен быть float")
        if p <= 0 or p > 1:
            raise ValueError("% должен быть в диапозоне: 0 < p <= 1")
        self.p = p

        self.path_to_dir = path_to_dir
        self.files = os.listdir(self.path_to_dir)
        self.path_to_new_dir_val = path_to_new_dir_val
        self.path_to_new_dir_train = path_to_new_dir_train



        # Создаем папки для сохранения
        if os.path.exists(path_to_new_dir_val):
            shutil.rmtree(path_to_new_dir_val)  # Удаляем папку, если она уже существует
        os.makedirs(path_to_new_dir_val, exist_ok=True)

        if os.path.exists(path_to_new_dir_train):
            shutil.rmtree(path_to_new_dir_train)  # Удаляем папку, если она уже существует
        os.makedirs(path_to_new_dir_train, exist_ok=True)

    def split(self):
        len_val = int(len(self.files)*self.p)
        len_train = len(self.files) - len_val
        with tqdm(total=len_val, desc="Отделение валидационных данных", unit="img") as pbar:
            for _ in range(len_val):
                idx = random.randint(0, len(self.files) - 1)
                file = self.files[idx]
                path_to_file = self.path_to_dir+f'/{file}'
                shutil.copy(path_to_file, self.path_to_new_dir_val+f'/{file}')
                self.files.pop(idx)

                pbar.update(1)

        with tqdm(total=len_train, desc="Отделение тренировочных данных", unit="img") as pbar:
            for _ in range(len_train):
                idx = random.randint(0, len(self.files) - 1)
                file = self.files[idx]
                path_to_file = self.path_to_dir+f'/{file}'
                shutil.copy(path_to_file, self.path_to_new_dir_train+f'/{file}')
                self.files.pop(idx)
                
                pbar.update(1)

        print(f"\nОтделено {len_val} изображений, для валидации в папку: {self.path_to_new_dir_val}")
        print(f"Отделено {len_train} изображений, для тренировки в папку: {self.path_to_new_dir_train}")



In [ ]:
class AugmentedDatasetSaver:
    def __init__(self, original_train_dataset, csv_path, augmentation_transform, new_train_dataset_path, output_csv_path, augment_count):
        """
        Создает и сохраняет аугментированный датасет из тренировочного.
        Также создает csv с ответами на аугментированный датасет, сохраняет ответы из train_answers

        original_train_dataset: Исходный датасет (torch Dataset).
        csv_path: путь к csv с ответами
        augmentation_transform: Трансформации для аугментации.
        new_train_dataset_path: Папка, куда будут сохранены данные.
        output_csv_path: Путь, куда будет сохранена csv
        augment_count: Количество аугментированных изображений, которые нужно создать.
        """
        self.original_train_dataset_path = original_train_dataset
        self.augmentation_transform = augmentation_transform
        self.new_train_dataset_path = new_train_dataset_path
        self.output_csv_path = output_csv_path
        self.original_labels = pd.read_csv(csv_path)

        
        if augment_count >= len(original_train_dataset):
            print("Введеный augment_count > длины датасета")
        self.augment_count = augment_count


        # Создаем папку для сохранения
        if os.path.exists(new_train_dataset_path):
            shutil.rmtree(new_train_dataset_path)  # Удаляем папку, если она уже существует
        os.makedirs(new_train_dataset_path, exist_ok=True)

    def save(self):
        new_entries = []
        max_id = self.original_labels['img_id'].max()
        with tqdm(total=self.augment_count, desc="Создание аугментированных данных", unit="img") as pbar:
            for i in range(self.augment_count):
                # Выбираем случайное изображение из оригинального датасета
                image, label, img_id = random.choice(self.original_train_dataset_path)

                # Преобразуем в тензор, если это PIL.Image
                if not isinstance(image, Image.Image):
                    raise ValueError(f"Unsupported image format: {type(image)}")

                # Применяем аугментацию
                augmented_image = self.augmentation_transform(image)

                

                # Генерируем имя файла, сохраняя оригинальное имя с префиксом
                img_id = max_id+i+1

                new_entries.append({'img_id': int(img_id), 'target_feature': label})

                img_save_path = os.path.join(self.new_train_dataset_path, f"{img_id}.jpg")

                # Преобразуем аугментированный тензор обратно в PIL.Image и сохраняем
                augmented_image = transforms.ToPILImage()(augmented_image)  # Перевод тензора в PIL
                augmented_image.save(img_save_path)

                pbar.update(1)
        # Создание DataFrame для новых данных
        augmented_df = pd.DataFrame(new_entries)
        combined_df = pd.concat([self.original_labels, augmented_df], ignore_index=True)
        combined_df.to_csv(self.output_csv_path, index=False)
        
        print(f"CSV файл с метками успешно сохранён в: {self.output_csv_path}")
        print(f"\nСохранено {self.augment_count} аугментированных изображений в папке: {self.new_train_dataset_path}")


In [ ]:
def merge_folders(path_train_dataset, path_augmented_only_dataset, path_destination_folder):
    """
    Объединяет две папки с картинками в одну. УДАЛЯЕТ папку path_augmented_only_dataset

    :param path_train_dataset: Путь к первой исходной папке
    :param path_augmented_only_dataset: Путь ко второй исходной папке
    :param path_destination_folder: Путь к папке назначения
    """
    # Создаем папку назначения, если она не существует
    if os.path.exists(path_destination_folder):
        shutil.rmtree(path_destination_folder)  # Удаляем папку, если она уже существует
    os.makedirs(path_destination_folder, exist_ok=True)
    
    # Функция для копирования файлов из папки
    def copy_files_from_folder(folder):
        with tqdm(total=len(os.listdir(folder)), desc=f"Копирование из {folder}", unit="img") as pbar:
            for file_name in os.listdir(folder):
                source_path = os.path.join(folder, file_name)
                dest_path = os.path.join(path_destination_folder, file_name)
                
                # Проверяем, является ли элемент файлом
                if os.path.isfile(source_path):
                    if os.path.exists(dest_path):
                        raise KeyError('файл с таким именем уже существует в папке назначения')
                    shutil.copy2(source_path, dest_path)
                pbar.update(1)
    
    # Копируем файлы из обеих папок
    copy_files_from_folder(path_train_dataset)
    copy_files_from_folder(path_augmented_only_dataset)
    #Удаляем папку с аугментацией
    shutil.rmtree(path_augmented_only_dataset)

По итогу получилось:   
папка img_train -- картинки в оригинале,  
папка train_dataset -- часть датасета, для обучения,  
папка val_dataset -- часть датасета, для валидации,   
папка augmented_and_train_dataset -- augmented_only_dataset+img_train,   
augmented_train_answers.csv -- ответы на все папки  


### Всё тот же кастомный резнет стал давать accuracy в целых 0.7 на 50 эпохе!
### Это значило, что пришло время первого сабмита, но тут нас ждало потресение. F1-score на каггле показал значение в 0.32411.
### Причин для такого могло быть много, от переобучения модели для val, до банального дисбаланса классов. В любом случае было решено исправить метрику для обучения на weighted F1-score, чтобы не повторять таких ситуций и не заходить в тупик
### Мы продолжали углубляться в вопрос различных архитектур. Была замечена интересная закономерность, при которой модели вплоть до 4 млн (в нашем **ResNet18** было 11.2 млн) параметров показывают скоры на порядок выше.
### В итоге мы протестировали архитектуры: **MobileNetV2**(3504872 параметров), **RegNetX16GF**(9190136 параметров), **RegNetY400mf**(4344144 параметров) и многие другие...
*Стоит подметить, что естественно все модели были **not pretrained***
### Настоящим золотым граалем среди архитектур оказался **EfficentNet**, помимо его выделяющегося F1-score со значением в 0.6 в базовой комплектации B0, он имел всего 4млн параметров, что давало гигансткие возможности для кастомизации. Мы так же попробовали и другие модели семейства, но их результат в связи с большим количеством параметров, подтвердив нашу теорию, оказался хуже.
### Чтобы понять, почему именно **EfficentNet** работает лучше и в дальнейшем маштабировать его, мы разобрали его архитектуру и выделили следующие особенности:
* SE-модули
* Эффективные MBConv-блоки с Depthwise и Pointwise свёртками
* Возможность легкого маштабирования с помощью Compound Scaling

In [ ]:
class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes=20):
        super(CustomEfficientNet, self).__init__()
        self.base_model = efficientnet_b0(pretrained=False)

        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(in_features, num_classes, bias=True)
        )

    def forward(self, x):
        return self.base_model(x)

def custom_efficientnet_b0(num_classes=20):
    return CustomEfficientNet(num_classes=num_classes)

### Данную модель мы стали использовать в качестве основной, первый же сабмит на каггл выдал F1 0.62395
### Один из участников стал искать способы повысить точность модель не меняя ее архитектуру, были опробованы:
* Различные виды loss-функций(Focal Loss, Cross-Entropy Loss...)
* Различные гиперпараметры(weight decay, learning rate...)
* Изменения аугментаций для датасета
* Различное количество бачей
### В то же время мы занимались кастомизацией самой модели. После активного "ковыряния" в архитектуре стало понятно, что внутренности модели лучше оставить в покое. Наращивание ее новыми блоками стало основным вектором дальнейшего развития. Мы добавляли свертки, skip-connectinos, даже внедряли Vision Transformer Layer. Итогом этого всего стала данная модель:

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes=20):
        super(CustomEfficientNet, self).__init__()
        self.base_model = models.efficientnet_b0(pretrained=False)
        self.extra_conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=1, stride=1),
            nn.ReLU(inplace=True)
        )

        self.base_model.features[0][0] = nn.Conv2d(64, 32, kernel_size=3, stride=2, padding=1)

        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        x = self.extra_conv(x)
        return self.base_model(x)

def custom_efficientnet_b0(num_classes=20):
    return CustomEfficientNet(num_classes=num_classes)

### Три дополнительных сверточных слоя с фильтрами размером 3×3, BatchNorm и ReLU добавляют способность модели извлекать более сложные локальные признаки на раннем этапе, Identity mapping позволяет усилить нелинейные преобразования. Основной фишкой этой модели стал высокий Dropout, он позволил нам иметь рост по скору на высоких эпохах, хоть и не всегда стабильный. Такая модель давала стабильно увеличивающийся скор вплоть до 0.65. При этом ее вариации по весам с разницей в 15-20 эпох имели около 1700 различных target_feature в сабмишнах. В теории это дает нам возможность ансамблировать несколько моделей и значительно увеличить скор.
### Нашим итоговым решение стал ансамбль из двух кастомных моделей на основе **ResNet18**, и модель на основе **EfficentNet-B0** на весах с разных эпох.


## ИТОГОВОЕ РЕШЕНИЕ

Для начала объявим модели, которые будут участвовать в ансамбле

In [ ]:
class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(nn.functional.softplus(x))

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        y = x.view(b, c, -1).mean(dim=2)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).view(b, c, 1, 1)
        return x * y

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None, dropout_prob=0.3):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.mish = Mish()
        self.group_norm1 = nn.GroupNorm(32, out_channels)
        self.dropout1 = nn.Dropout2d(p=dropout_prob)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.group_norm2 = nn.GroupNorm(32, out_channels)
        self.dropout2 = nn.Dropout2d(p=dropout_prob)

        self.se_block = SEBlock(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.mish(out)
        out = self.group_norm1(out)
        out = self.dropout1(out)

        out = self.conv2(out)
        out = self.mish(out)
        out = self.group_norm2(out)
        out = self.dropout2(out)

        out = self.se_block(out)
        out += identity
        out = self.mish(out)
        return out

class UpdatedResNet(nn.Module):
    def __init__(self, block, layers, num_classes=20, dropout_prob=0.3):
        super(UpdatedResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.mish = Mish()
        self.group_norm = nn.GroupNorm(32, 64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0], dropout_prob=dropout_prob)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dropout_prob=dropout_prob)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dropout_prob=dropout_prob)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dropout_prob=dropout_prob)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1, dropout_prob=0.3):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.GroupNorm(32, out_channels)
            )
        
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample, dropout_prob=dropout_prob))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels, dropout_prob=dropout_prob))
        
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.mish(x)
        x = self.group_norm(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

def custom_resnet18(num_classes=20, dropout_prob=0.3):
    return UpdatedResNet(ResidualBlock, [2, 2, 2, 2], num_classes, dropout_prob)

In [ ]:
model_res = custom_resnet18(dropout_prob=0.5).to(device)
model_res_2 = custom_resnet18(dropout_prob=0.5).to(device)

In [ ]:
from torchvision.models import efficientnet_b0
class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes=20):
        super(CustomEfficientNet, self).__init__()
        # Load the base EfficientNet-B0 architecture without pretrained weights
        self.base_model = efficientnet_b0(pretrained=False)

        # Replace the last classifier layer for 20 classes
        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(in_features, num_classes, bias=True)  # Explicitly set bias=True
        )

    def forward(self, x):
        return self.base_model(x)

def custom_efficientnet_b0(num_classes=20):
    return CustomEfficientNet(num_classes=num_classes)



In [ ]:
model_effnb0 = custom_efficientnet_b0(num_classes=20)

In [ ]:
class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes=20):
        super(CustomEfficientNet, self).__init__()

        # Загрузка базовой модели EfficientNet-B0 без предобученных весов
        self.base_model = efficientnet_b0(pretrained=False)

        # Добавление дополнительных сверточных слоев и Batch Normalization
        # Сначала добавляем дополнительные сверточные слои и Batch Normalization
        self.extra_conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=1, stride=1),  # Identity mapping
            nn.ReLU(inplace=True)
        )


        # Обновляем первый слой базовой модели для использования 64 каналов (после добавленных слоев)
        self.base_model.features[0][0] = nn.Conv2d(64, 32, kernel_size=3, stride=2, padding=1)

        # Заменяем последний классификатор для 20 классов
        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=True),  # Увеличиваем dropout
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        # Применяем дополнительные слои перед базовой моделью
        x = self.extra_conv(x)
        
        # Дальше передаем результат в базовую модель
        return self.base_model(x)

def custom_efficientnet_b0(num_classes=20):
    return CustomEfficientNet(num_classes=num_classes)

In [ ]:
model_custom_effn_401 = custom_efficientnet_b0(num_classes=20)
model_custom_effn_300 = custom_efficientnet_b0(num_classes=20)

После объявления всех моделей загружаем веса (Лежат на [google drive](https://drive.google.com/drive/folders/18pqO6QCQjDEGPZ28Z5VcWkbDqmdSGKe6?usp=sharing))

In [ ]:
path_custom_effn_401 = './train_info/checkpoints_17_12_19_24/best_checkpoint_251_val_F1=0.6286.pt'
path_custom_effn_300 = './train_info/checkpoints_17_12_19_24/best_checkpoint_142_val_F1=0.6188.pt'
path_res_2 = './train_info/checkpoints_11_12_7_1/best_checkpoint_40_val_accuracy=0.6990.pt'
path_effnb0 = './train_info/check_effnb0/best_checkpoint_18_val_F1=0.5903.pt'
path_res_check = './train_info/checkpoints_14_12_17_44/best_UpdatedResNet_checkpoint_22_val_accuracy=0.6505.pt'

In [ ]:
checkpoint_custom_effn_401 = torch.load(path_custom_effn_401, map_location=torch.device(device))
checkpoint_custom_effn_300 = torch.load(path_custom_effn_300, map_location=torch.device(device))
checkpoint_res_2  = torch.load(path_res_2, map_location=torch.device(device))
checkpoint_effnb0  = torch.load(path_effnb0, map_location=torch.device(device))
checkpoint_res = torch.load(path_res_check, map_location=torch.device(device))

model_custom_effn_401.load_state_dict(checkpoint_custom_effn_401["model"])
model_custom_effn_300.load_state_dict(checkpoint_custom_effn_300["model"])
model_res_2.load_state_dict(checkpoint_res_2["model"])
model_effnb0.load_state_dict(checkpoint_effnb0["model"])
model_res.load_state_dict(checkpoint_res["model"])

model_custom_effn_401.eval()
model_custom_effn_300.eval()
model_res_2.eval()
model_effnb0.eval()
model_res.eval()

In [ ]:
# Усредненный ансамбль
def ensemble_predict(dataloader, model_1, model_2, model_3, model_4,model_5, device='cuda'):
    model_1.to(device)
    model_2.to(device)
    model_3.to(device)
    model_4.to(device)
    model_5.to(device)

    predictions = []
    images = []
    labels = []

    with torch.no_grad():
        for batch in dataloader:
            inputs, targets, _ = batch  # inputs - изображения, targets - метки
            inputs = inputs.to(device)

            # Получаем вероятности от обеих моделей
            probs_1 = F.softmax(model_1(inputs), dim=1)
            probs_2 = F.softmax(model_2(inputs), dim=1)
            probs_3 = F.softmax(model_3(inputs), dim=1)
            probs_4 = F.softmax(model_4(inputs), dim=1)
            probs_5 = F.softmax(model_5(inputs), dim=1)

            # Усредняем вероятности
            ensemble_probs = (probs_1 + probs_2 + probs_3 + probs_4+ probs_5) / 5

            # Выбираем класс с максимальной вероятностью
            predicted_classes = torch.argmax(ensemble_probs, dim=1)
            predictions.extend(predicted_classes.cpu().numpy())

            # Сохраняем изображения и метки для визуализации
            images.extend(inputs.cpu())
            labels.extend(targets.cpu().numpy())

    return predictions, images, labels

In [ ]:
from numpy import sqrt


def visualize_predictions(images, labels, predictions, class_names,  num_images= 9):
    """
    Визуализирует изображения с реальными и предсказанными метками.

    Args:
        images (list): Список изображений.
        labels (list): Реальные метки.
        predictions (list): Предсказанные метки.
        class_names (dict): Словарь с именами классов, где ключ - метка, значение - имя класса.
    """
    plt.figure(figsize=(12, 12))
    for i in range(num_images):
        plt.subplot(int(sqrt(num_images)), int(sqrt(num_images)), i + 1)
        plt.imshow(images[i].permute(1, 2, 0))  # Преобразуем изображение для отображения
        plt.title(f"Real: {class_names[labels[i]]}\nPred: {class_names[predictions[i]]}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.metrics import f1_score

def calculate_f1(predictions, labels):
    """
    Вычисляет F1-меру для предсказаний ансамбля.

    Args:
        predictions (list): Список предсказанных классов.
        labels (list): Список реальных меток.

    Returns:
        float: Значение F1-меры.
    """
    return f1_score(labels, predictions, average='weighted')

In [ ]:
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)

Прогоняем через датасет через ансамбль и высчитываем f1, он не равен тому, что на kaggle, но отражает ситуацию, что позволяет не тратить лишине сабмиты

In [ ]:
ensemble_predictions, images, labels = ensemble_predict(val_loader, model_custom_effn_401, model_custom_effn_300, model_effnb0,model_res_2, model_res)
f1 = calculate_f1(ensemble_predictions, labels)
print(f"f1: {f1:.2%}")

Код, для создания csv


In [ ]:
test_images_path = "../data/img_test"

transform = transforms.Compose([
    transforms.Resize((227, 227)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_image_files = [f for f in os.listdir(test_images_path) if f.endswith('.jpg') or f.endswith('.png')]

results = []

model_1 = model_custom_effn_401
model_2 = model_custom_effn_300
model_3 = model_effnb0
model_4 = model_res
model_5 = model_res_2
# model_6 = model_custom_effn_300

for image_file in test_image_files:
    image_path = os.path.join(test_images_path, image_file)
    image = Image.open(image_path).convert("RGB") 
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        probs_1 = F.softmax(model_1(image), dim=1)
        probs_2 = F.softmax(model_2(image), dim=1)
        probs_3 = F.softmax(model_3(image), dim=1)
        probs_4 = F.softmax(model_4(image), dim=1)
        probs_5 = F.softmax(model_5(image), dim=1)
        # probs_6 = F.softmax(model_6(image), dim=1)

        # Усредняем вероятности
        # ensemble_probs = (probs_1 + probs_2 + probs_3 + probs_4+probs_5+probs_6) / 6
        ensemble_probs = (probs_1 + probs_2 + probs_3 + probs_4+probs_5) / 5

        # Находим класс с максимальной вероятностью
        predicted_class = torch.argmax(ensemble_probs, dim=1).item()

    image_id = os.path.splitext(image_file)[0]
    results.append({"id": image_id, "target_feature": predicted_class})

submission_df = pd.DataFrame(results)

submission_file_path = "./submission.csv"
submission_df.to_csv(submission_file_path, index=False)

print(f"Submission file saved to: {submission_file_path}")


## ВЫВОДЫ

###
### Возможности развития данной модели - очень велики. Мы несомненно получили один из лучших скоров на курсе с моделью, которая имеет всего 4074512 параметров. Очевидно и то, что ее маштабирование в условиях реальной задачи представляется во много раз более легким, чем того же ResNet18.
### Итого на эту работу у нас ушло около трех лимитов Kaggle по GPU, бесчисленное количество ночного обучения на локальных мощностях и 60ГБ на диске для весов моделей. Мы рады, что смогли показать достойный результат, несмотря на все обстоятельства.